In [1]:
import os
import sys
import warnings
import pandas as pd
from tqdm.notebook import tqdm

base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"base_path: {base_path}")
sys.path.append(base_path)

base_path: /home/uoscisai/Experiments/Football/sr-press


In [2]:
from express.databases import SQLiteDatabase
from express.datasets import PressingDataset

from express import features as fs
from express import labels as ls

In [3]:
TRAIN_DB_PATH = os.path.join(base_path, "stores/train_database.sqlite")
TEST_DB_PATH = os.path.join(base_path, "stores/test_database.sqlite")

train_db = SQLiteDatabase(TRAIN_DB_PATH)
test_db = SQLiteDatabase(TEST_DB_PATH)

print("train_db:", train_db)
print("test_db:", test_db)

train_db: <express.databases.sqlite.SQLiteDatabase object at 0x7b3af7c89240>
test_db: <express.databases.sqlite.SQLiteDatabase object at 0x7b3bf998b040>


In [4]:
test_db.games()[16:32]

,season_id,competition_id,game_day,game_date,home_team_id,away_team_id,competition_stage,home_score,away_score,venue,referee
game_id,,,,,,,,,,,
3869117,106,43,4,2022-12-03 17:00:00,941,1839,Round of 16,3,1,Sheikh Khalifa International Stadium,Wilton Pereira Sampaio
3869118,106,43,4,2022-12-04 21:00:00,768,787,Round of 16,3,0,Al Bayt Stadium,Ivan Arcides Barton Cisneros
3869151,106,43,4,2022-12-03 21:00:00,779,792,Round of 16,2,1,Ahmad bin Ali Stadium,Szymon Marciniak
3869152,106,43,4,2022-12-04 17:00:00,771,789,Round of 16,3,1,Al Thumama Stadium,Jesús Valenzuela Sáez
3869219,106,43,4,2022-12-05 17:00:00,778,785,Round of 16,1,1,Al Janoub Stadium,Ismail Elfath
3869220,106,43,4,2022-12-06 17:00:00,788,772,Round of 16,0,0,Education City Stadium,Fernando Andrés Rapallini
3869253,106,43,4,2022-12-05 21:00:00,781,791,Round of 16,4,1,Stadium 974,Clément Turpin
3869254,106,43,4,2022-12-06 21:00:00,780,773,Round of 16,6,1,Lusail Stadium,César Arturo Ramos Palazuelos
3869321,106,43,5,2022-12-09 21:00:00,941,779,Quarter-finals,2,2,Lusail Stadium,Antonio Miguel Mateu Lahoz


In [5]:
print(train_db.games().shape, test_db.games().shape)

(151, 11) (49, 11)


In [6]:
all_features = [f.__name__ for f in fs.all_features]
all_labels = [f.__name__ for f in ls.all_labels]
print("Features:", all_features)
print("Labels:", all_labels)

Features: ['actiontype', 'actiontype_onehot', 'result', 'result_onehot', 'bodypart', 'bodypart_onehot', 'time', 'startlocation', 'relative_startlocation', 'endlocation', 'relative_endlocation', 'startpolar', 'endpolar', 'movement', 'team', 'time_delta', 'space_delta', 'goalscore', 'angle', 'under_pressure', 'packing_rate', 'ball_height_onehot', 'speed', 'freeze_frame_360', 'nb_opp_in_path', 'dist_opponent', 'defenders_in_3m_radius', 'closest_3_players', 'closest_11_players', 'expected_3_receiver_and_presser_by_distance']
Labels: ['concede_shots', 'counterpress']


In [9]:
train_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "datasets", "train"), 
    xfns=["startlocation", "closest_11_players", 'actiontype', "freeze_frame_360"],
    yfns=["counterpress"], 
    load_cached =False,
    nb_prev_actions = 3,
)

test_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "datasets", "test"), 
    xfns=["startlocation", "closest_11_players", 'actiontype', "freeze_frame_360"],
    yfns=["counterpress"], 
    load_cached =False,
    nb_prev_actions = 3,
)

In [10]:
train_dataset.create(train_db)
test_dataset.create(test_db)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [00:05<00:00,  8.19it/s]


In [11]:
train_dataset.features

start_x_a0  start_y_a0  start_x_a1  start_y_a1  start_x_a2  \
game_id action_id                                                               
3788741 3             41.7375      61.285     73.7625      25.415     77.0000   
        19            84.5250      59.500     41.4750      59.500     38.5875   
        27            27.2125      13.940     28.6125       9.945     32.8125   
        31            75.7750      60.690     79.1000      58.480     79.7125   
        35            79.9750      58.055     84.0875      57.290     75.7750   
...                       ...         ...         ...         ...         ...   
3943043 2283          82.0750      20.230     67.9000       0.000      4.2875   
        2286          34.3000      61.710     71.6625      67.575     13.0375   
        2302          47.7750      31.790     69.6500      55.930     66.4125   
        2304          49.6125      35.700     47.7750      31.790     47.7750   
        2309           8.6625      64.260     11.2875      65.535     12.0750   

                   start_y_a2  teammate_1_x_a0  teammate_1_y_a0  \
game_id action_id                                                 
3788741 3              24.055        53.579218        52.737630   
        19             54.400        76.496764        53.723000   
        27              0.000        21.941713        14.556432   
        31             56.185        73.332487        65.151690   
        35             60.690        78.241322        57.741576   
...                       ...              ...              ...   
3943043 2283           31.195        67.832653        16.084311   
        2286           47.345        42.269977        58.389380   
        2302           56.440        50.047145        37.296331   
        2304           31.790        49.390287        32.930291   
        2309           50.490        10.449531        59.027809   

                   teammate_1_distance_a0  teammate_2_x_a0  ...  \
game_id action_id                                           ...   
3788741 3                       14.604240        34.377131  ...   
        19                       9.890718        65.244864  ...   
        27                       5.306711        29.895983  ...   
        31                       5.086506        79.741558  ...   
        35                       1.761781        81.094272  ...   
...                                   ...              ...  ...   
3943043 2283                    14.833448        58.134253  ...   
        2286                     8.634063        41.571906  ...   
        2302                     5.956704        41.740027  ...   
        2304                     2.778608        42.628292  ...   
        2309                     5.528951        14.996899  ...   

                   opponent_10_distance_a2  opponent_11_x_a2  \
game_id action_id                                              
3788741 3                        42.589874               NaN   
        19                             NaN               NaN   
        27                             NaN               NaN   
        31                             NaN               NaN   
        35                             NaN               NaN   
...                                    ...               ...   
3943043 2283                           NaN               NaN   
        2286                           NaN               NaN   
        2302                           NaN               NaN   
        2304                           NaN               NaN   
        2309                     24.951294               NaN   

                   opponent_11_y_a2  opponent_11_distance_a2  type_id_a0  \
game_id action_id                                                          
3788741 3                       NaN                      NaN          23   
        19                      NaN                      NaN          23   
        27                      NaN                      NaN          23   
        

In [ ]:
test_dataset.features['freeze_frame_360_a0'].iloc[0]

In [ ]:
train_dataset.labels

In [ ]:
test_dataset.labels

In [ ]:
train_dataset.labels["counterpress"].value_counts()

In [ ]:
test_dataset.labels["counterpress"].value_counts()